In [1]:
char_list = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
char_dict = {i: char_list[i] for i in range(len(char_list))}

print(char_dict)

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}


In [ ]:
!pip install mediapipe
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

In [5]:
import os

path="/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data"

In [6]:
import cv2

X=[]
Y=[]
label=[]

k=0

for i in char_list:
    path1=os.path.join(path,i)
    for j in os.listdir(path1):
        image_path = os.path.join(path1, j)
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                x1=[]
                y1=[]
                for idx, landmark in enumerate(hand_landmarks.landmark):
                    h, w, _ = image.shape
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    x1.append(x)
                    y1.append(y)
                X.append(x1)
                Y.append(y1)
                label.append(k)
        else:
            continue
    k=k+1

W0000 00:00:1751174468.596762     107 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create feature vectors by keeping X and Y separate
features = [x + y for x, y in zip(X, Y)]  # x: list of X coords, y: list of Y coords

# Create DataFrame
df = pd.DataFrame(features)
df['label'] = label  # Append label as the target column

# Split into features and labels
X_df = df.drop('label', axis=1)
y_df = df['label']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Train Random Forest model
model = RandomForestClassifier()
model.fit(x_train, y_train)

# Predict and evaluate
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9744245524296675


In [43]:
import cv2
import mediapipe as mp

def predict_asl_letter(model, image_path, char_dict):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=True, 
        max_num_hands=1,
        min_detection_confidence=0.1,
        min_tracking_confidence=0.1
    )
    
    image = cv2.imread(image_path)
    if image is None:
        return "Error: Could not load image"
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            x_coords = []
            y_coords = []
            
            for landmark in hand_landmarks.landmark:
                h, w, _ = image.shape
                x, y = int(landmark.x * w), int(landmark.y * h)
                x_coords.append(x)
                y_coords.append(y)
            
            features = x_coords + y_coords
            prediction = model.predict([features])[0]
            predicted_letter = char_dict[prediction]
            
            return predicted_letter
    else:
        return "No hand detected"

# Test with your image
image_path = "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/N/11.jpg"

# Your char_dict from training
char_list = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
char_dict = {i: char_list[i] for i in range(len(char_list))}

result = predict_asl_letter(model, image_path, char_dict)
print(result)

N


W0000 00:00:1751177405.017764     382 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751177405.069800     382 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [49]:
import cv2
import mediapipe as mp
import os
import warnings
import logging
import sys
from contextlib import redirect_stderr
from io import StringIO

# Suppress all warnings and logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GLOG_minloglevel'] = '3'
warnings.filterwarnings('ignore')
logging.getLogger().setLevel(logging.ERROR)

def predict_asl_letter(model, image_path, char_dict):
    mp_hands = mp.solutions.hands
    
    # Suppress stderr during MediaPipe operations
    with redirect_stderr(StringIO()):
        hands = mp_hands.Hands(
            static_image_mode=True, 
            max_num_hands=1,
            min_detection_confidence=0.1,
            min_tracking_confidence=0.1
        )
        
        image = cv2.imread(image_path)
        if image is None:
            return "?"
        
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            x_coords = []
            y_coords = []
            
            for landmark in hand_landmarks.landmark:
                h, w, _ = image.shape
                x, y = int(landmark.x * w), int(landmark.y * h)
                x_coords.append(x)
                y_coords.append(y)
            
            features = x_coords + y_coords
            prediction = model.predict([features])[0]
            predicted_letter = char_dict[prediction]
            
            return predicted_letter
    else:
        return "?"

def predict_word_from_images(model, image_paths, char_dict):
    word = ""
    for image_path in image_paths:
        letter = predict_asl_letter(model, image_path, char_dict)
        word += letter
    return word

# Your char_dict from training
char_list = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
char_dict = {i: char_list[i] for i in range(len(char_list))}

# Example with list of image paths
image_paths = [
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/H/11.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/E/105.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/L/11.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/L/10.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/O/103.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/E/105.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/V/103.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/E/109.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/R/104.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/Y/104.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/O/103.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/N/104.jpg",
    "/kaggle/input/american-sign-language-asl/American Sign Language (ASL)/data/E/104.jpg",
]

result = predict_word_from_images(model, image_paths, char_dict)
print(result)

W0000 00:00:1751177836.424763     521 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751177836.475612     521 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751177836.561916     525 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751177836.606460     523 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751177836.696625     529 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751177836.745019     527 inference_feedback_manager.cc:114] Feedback manager 

HELLOEVERYONE


W0000 00:00:1751177838.053068     569 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751177838.099921     569 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
